In [90]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import plotly.express as px
import emd
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 3
WINDOW_SIZE = 100
SLIDING_STEP = int(WINDOW_SIZE * 0.4)
KEY_CLASS = {0:'undefined action', 1:'up', 2:'down', 3:'left', 4:'right', 5:'quick touch'}
CLASS_NUMBER = 5 # 0 is not a class
NUM_IMF = 3
LABEL_THRESHOLD = 0.65
BELIEF_THRESHOLD = 0.7
INITIAL_PULSE = 100 # abandon initial pulse data

if CLASS_NUMBER < 2:
    CLASS_NUMBER = 2

In [80]:
def slicing(x, y):
    totalLength = x.shape[0]
    assert totalLength == y.shape[0], "Data numbers not matching with that of labels."
    if totalLength <= WINDOW_SIZE:
        return x, y

    y = one_hot(y)
    
    thresholdWindow = LABEL_THRESHOLD * WINDOW_SIZE
    retx = None
    rety = None
    retUnknown = None
    
    i = 0
    while (totalLength - i) > WINDOW_SIZE:
        new = (x[i:(i + WINDOW_SIZE), :])[np.newaxis, :]
        
        classSum = np.sum(y[i:(i + WINDOW_SIZE)], axis = 0)
        maxIdx = np.argmax(classSum)
        if classSum[maxIdx] > thresholdWindow:
            if not isinstance(retx, np.ndarray):
                retx = new.copy()
                rety = [maxIdx + 1]
            else:
                retx = np.concatenate([retx, new], axis=0)
                rety.append(maxIdx + 1)
        else:
            if not isinstance(retUnknown, np.ndarray):
                retUnknown = new.copy()
            else:
                retUnknown = np.concatenate([retUnknown, new], axis=0)

        i += SLIDING_STEP
        
    return retx, one_hot(rety), retUnknown

def one_hot(arr):
    ret = []
    for val in arr:
        tmp = [0] * CLASS_NUMBER
        if val > 0:
            tmp[val - 1] = 1
        ret.append(np.array(tmp))
        
    return np.array(ret)    

def emdSignal(sig):
    dataNumber = sig.shape[0]
    channel = sig.shape[-1]
    ret = None
    
    for i in range(dataNumber):
        temp = None
        
        for c in range(channel):
            raw = sig[i, :, c]
            imf = emd.sift.sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(temp) == np.ndarray: 
                temp = imf
            else: 
                temp = np.concatenate([temp, imf], axis = 1)
            
        if type(temp) == np.ndarray: 
            if not type(ret) == np.ndarray: 
                ret = temp[np.newaxis, :]
            else: 
                ret = np.concatenate([ret, temp[np.newaxis, :]], axis = 0)
                
    return ret

def buildModel(shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = shape),
        tf.keras.layers.Conv1D(int(WINDOW_SIZE * 0.4 // 3), int(WINDOW_SIZE * 0.5 // 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(int(WINDOW_SIZE * 0.5 // 3), int(WINDOW_SIZE * 0.6 // 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(int(WINDOW_SIZE * 0.6 // 3), int(WINDOW_SIZE / 1.2 * 0.7 // 3), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense((WINDOW_SIZE // 100) * 256, activation='relu'),
        tf.keras.layers.Dense(CLASS_NUMBER, activation='softmax')]
    )
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.keras.metrics.CategoricalAccuracy()])
                        #  tf.keras.metrics.Precision(thresholds = 0.5),
                        #  tf.keras.metrics.Recall(thresholds= 0.5)])
    return model

In [81]:
#Load, preprocess and split record files
trainSignalFiles = ["2023_Mar_29_170717_l5m6r7_record_X", "2023_Mar_29_191038_l5m6r7_record_X"]
trainLabelFiles = [x[:-1] + 'y' for x in trainSignalFiles]


numX = 0
numUX = 0
sigPLot = None
trainSignal, trainLabel, unknownActions = [], [], []
for sfp, lfp in zip(trainSignalFiles, trainLabelFiles):
    tempSig = np.load("./data/" + sfp + ".npy")[INITIAL_PULSE:]
    tempLbl = np.load("./data/" + lfp + ".npy")[INITIAL_PULSE:]
    sigPLot = tempSig if not isinstance(sigPLot, np.ndarray) else np.concatenate([sigPLot, tempSig], axis=0)
    X, y, X_unknown = slicing(tempSig, tempLbl)
    trainSignal.append(X)
    trainLabel.append(y)
    unknownActions.append(X_unknown)
    numX += X.shape[0]
    numUX += X_unknown.shape[0]

print("Number of X: {}, unknown X: {}".format(numX, numUX))

Number of X: 932, unknown X: 1118


In [ ]:
px.line(sigPLot)

In [82]:
X_train = None
X_test = None
y_train = None
y_test = None
for x, y, u in zip(trainSignal, trainLabel, unknownActions):
    if x.shape[0] // CLASS_NUMBER < u.shape[0]:
        X_unknown_add = u[np.random.choice(u.shape[0], size = int(x.shape[0] // CLASS_NUMBER), replace = False)]
        y_unknown_add = (np.random.rand(int(x.shape[0] // CLASS_NUMBER), CLASS_NUMBER) * 0.1 + 0.15)
    else:
        X_unknown_add = u
        y_unknown_add = (np.random.rand(u.shape[0], CLASS_NUMBER) * 0.1 + 0.15)

    Xt = np.concatenate([x, X_unknown_add], axis = 0)
    yt = np.concatenate([y, y_unknown_add], axis = 0)
        
    xtr, xte, ytr, yte = train_test_split(Xt, yt, test_size=0.2, random_state=343)
    if not isinstance(X_train, np.ndarray):
        X_train = xtr
        X_test = xte
        y_train = ytr
        y_test = yte
    else:
        X_train = np.concatenate([X_train, xtr], axis=0)
        X_test = np.concatenate([X_test, xte], axis=0)
        y_train = np.concatenate([y_train, ytr], axis=0)
        y_test = np.concatenate([y_test, yte], axis=0)

print(X_train.shape, X_test.shape, X_unknown.shape)

(892, 100, 3) (225, 100, 3) (494, 100, 3)


In [83]:
#Empirical Mode Decomposotion
X_train = emdSignal(X_train)
X_test = emdSignal(X_test)

In [84]:
#Unknown actions & EMD
X_unknown = None
for u in unknownActions:
    if not isinstance(X_unknown, np.ndarray):
        X_unknown = u
    else:
        X_unknown = np.concatenate([X_unknown, u], axis=0)
X_unknown = emdSignal(X_unknown)

In [85]:
#Combine new signal with processed signal
pTrainSignalFiles = ["230326_X_train"]
pTrainLabelFiles = [x.replace('X', 'y') for x in pTrainSignalFiles]
pTestSignalFiles = ["230326_X_test"]
pTestLabelFiles = [x.replace('X', 'y') for x in pTestSignalFiles]
pUnknownSignalFiles = ["230326_X_Unknown"]

if len(pTrainSignalFiles):
    for ptsf, ptlf in zip(pTrainSignalFiles, pTrainLabelFiles):
        X_train = np.concatenate([X_train, np.load("./data/emd/" + ptsf + ".npy")])
        y_train = np.concatenate([y_train, np.load("./data/emd/" + ptlf + ".npy")])
    for ptsf, ptlf in zip(pTestSignalFiles, pTestLabelFiles):
        X_test = np.concatenate([X_test, np.load("./data/emd/" + ptsf + ".npy")])
        y_test = np.concatenate([y_test, np.load("./data/emd/" + ptlf + ".npy")])
    for pusf in pUnknownSignalFiles:
        X_unknown = np.concatenate([X_unknown, np.load("./data/emd/" + pusf + ".npy")])

In [10]:
#Save processed signal
np.save("./data/emd/230326_X_train", X_train)
np.save("./data/emd/230326_y_train", y_train)
np.save("./data/emd/230326_X_test", X_test)
np.save("./data/emd/230326_y_test", y_test)
np.save("./data/emd/230326_X_unknown", X_unknown)

In [86]:
#Training
model = buildModel(X_train.shape[1:])
model.summary()
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=BATCH_SIZE,
                    epochs=50,
                    validation_data=[X_test, y_test])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 100, 13)           1885      
                                                                 
 batch_normalization_12 (Bat  (None, 100, 13)          400       
 chNormalization)                                                
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 50, 13)           0         
 g1D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 50, 13)            0         
                                                                 
 conv1d_13 (Conv1D)          (None, 50, 16)            4176      
                                                                 
 batch_normalization_13 (Bat  (None, 50, 16)          

In [87]:
f1 = np.array(history.history['loss']).flatten()
valf1 = np.array(history.history['val_loss']).flatten()
px.line(pd.DataFrame(np.array([f1, valf1]).T, columns=['loss', 'val_loss'])).show()

In [ ]:
model = tf.keras.models.load_model('./model/MTJaw0329')

In [88]:
All_X = np.concatenate([X_test, X_unknown])
All_y = np.concatenate([y_test, np.array([[0] * CLASS_NUMBER for _ in range(X_unknown.shape[0])])])
res = model(All_X)

In [91]:
# pd.DataFrame([KEY_CLASS[np.argmax(r) + 1] if any(r) else KEY_CLASS[0] for r in All_y]).value_counts()
# pd.DataFrame([KEY_CLASS[np.argmax(r) + 1 if r[np.argmax(r)] > BELIEF_THRESHOLD else 0] for r in res]).value_counts()
gj = {}
bj = {}
tg = 0
tb = 0

for key, cls in KEY_CLASS.items():
    gj[cls] = 0
    bj[cls] = 0
    
for r, p in zip(All_y, res):
    rm = KEY_CLASS[np.argmax(r) + 1 if any(r) else 0]
    pm = KEY_CLASS[np.argmax(p) + 1 if p[np.argmax(p)] > BELIEF_THRESHOLD else 0]
    if rm == pm:
        tg += 1
        gj[rm] += 1
    else:
        tb += 1
        bj[rm] += 1
        
for key, cls in KEY_CLASS.items():
    print("Action: {}, True: {}, False: {}, Accuracy: {:.4f}".format(cls, gj[cls], bj[cls], gj[cls] / (gj[cls] + bj[cls])))
print("Left&Right True: {}, False: {}, Accuracy: {:.4f}".format(gj["left"] + gj["right"], bj["left"] + bj["right"], (gj["left"] + gj["right"]) / (gj["left"] + gj["right"] + bj["left"] + bj["right"])))
print("Total True: {}, False: {}, Accuracy: {:.4f}".format(tg, tb, tg / (tg + tb)))

Action: undefined action, True: 4630, False: 1378, Accuracy: 0.7706
Action: up, True: 58, False: 95, Accuracy: 0.3791
Action: down, True: 20, False: 112, Accuracy: 0.1515
Action: left, True: 298, False: 206, Accuracy: 0.5913
Action: right, True: 214, False: 215, Accuracy: 0.4988
Action: quick touch, True: 20, False: 70, Accuracy: 0.2222
Left&Right True: 512, False: 421, Accuracy: 0.5488
Total True: 5240, False: 2076, Accuracy: 0.7162


In [ ]:
class GA():
    def __init__(self, popSize = 100, maxGen = 100, mr = 0.1, cr = 0.8):
        self.popSize = popSize
        self.pop = np.random.random(popSize)
        self.maxGen = maxGen
        self.cr = cr
        self.mr = mr

    def crossover(self, pa, pb):
        if (np.random.random() < self.cr):
            alpha = np.clip(np.random.random(), 0.1, 0.9)
            oa = pa * alpha + pb * (1 - alpha)
            ob = pb * alpha + pa * (1 - alpha)
            return oa, ob
        else:
            return pa, pb
    
    def mutation(self, i):
        pass
    
def objective(belief):
    gj = {}
    bj = {}
    tg = 0
    tb = 0

    for key, cls in KEY_CLASS.items():
        gj[cls] = 0
        bj[cls] = 0
        
    for r, p in zip(All_y, res):
        rm = KEY_CLASS[np.argmax(r) + 1 if any(r) else 0]
        pm = KEY_CLASS[np.argmax(p) + 1 if p[np.argmax(p)] > belief else 0]
        if rm == pm:
            tg += 1
            gj[rm] += 1
        else:
            tb += 1
            bj[rm] += 1

    undef_acc = gj["undefined action"] / (gj["undefined action"] + bj["undefined action"])
    left_acc = gj["left"] / (gj["left"] + bj["left"])
    right_acc = gj["right"] / (gj["right"] + bj["right"])
    return undef_acc + left_acc + right_acc


In [48]:
model.save("./model/MTJaw0329", save_format="tf")

### Other test

In [9]:
testSignalFiles = ["2023_Mar_29_170717_l5m6r7_record_X"]
testLabelFiles = ["2023_Mar_29_170717_l5m6r7_record_y"]

testSignal, testLabel = None, None
for sfp, lfp in zip(testSignalFiles, testLabelFiles):
    testSignal = np.load("./data/" + sfp + ".npy")[INITIAL_PULSE:] if not isinstance(testSignal, np.ndarray) else np.concatenate([testSignal, np.load("./data/" + sfp + ".npy")[INITIAL_PULSE:]], axis=0)
    testLabel = np.load("./data/" + lfp + ".npy")[INITIAL_PULSE:] if not isinstance(testLabel, np.ndarray) else np.concatenate([testLabel, np.load("./data/" + lfp + ".npy")[INITIAL_PULSE:]], axis=0)

X2, y2, X2_unknown = slicing(testSignal, testLabel)
print("Number of X: {}, unknown X: {}".format(X2.shape[0], X2_unknown.shape[0]))

Number of X: 393, unknown X: 624


In [10]:
X2 = emdSignal(X2)
# X2_unknown = emdSignal(X2_unknown)
# X2t = np.concatenate([X2, X2_unknown], axis=0)
# y2t = np.concatenate([y2, np.zeros((X2_unknown.shape[0], CLASS_NUMBER))])

In [26]:
res2 = model(X2)
gj = {}
bj = {}
tg = 0
tb = 0

for key, cls in KEY_CLASS.items():
    gj[cls] = 0
    bj[cls] = 0
    
for r, p in zip(y2, res2):
    rm = KEY_CLASS[np.argmax(r) + 1 if any(r) else 0]
    pm = KEY_CLASS[np.argmax(p) + 1 if p[np.argmax(p)] > BELIEF_THRESHOLD else 0]
    if rm == pm:
        tg += 1
        gj[rm] += 1
    else:
        tb += 1
        bj[rm] += 1
        
for key, cls in KEY_CLASS.items():
    if ((gj[cls] + bj[cls]) > 0):
        print("Action: {}, True: {}, False: {}, Accuracy: {:.4f}".format(cls, gj[cls], bj[cls], gj[cls] / (gj[cls] + bj[cls])))
print("Total True: {}, False: {}, Accuracy: {:.4f}".format(tg, tb, tg / (tg + tb)))

Action: left, True: 25, False: 179, Accuracy: 0.1225
Action: right, True: 48, False: 141, Accuracy: 0.2540
Total True: 73, False: 320, Accuracy: 0.1858


### Backup code

In [ ]:
if X.shape[0] // CLASS_NUMBER < X_unknown.shape[0]:
    X_unknown_add = X_unknown[np.random.choice(X_unknown.shape[0], size = int(X.shape[0] // CLASS_NUMBER), replace = False)]
    y_unknown_add = (np.random.rand(int(X.shape[0] // CLASS_NUMBER), CLASS_NUMBER) * 0.1 + 0.15)
else:
    X_unknown_add = X_unknown
    y_unknown_add = (np.random.rand(X_unknown.shape[0], CLASS_NUMBER) * 0.1 + 0.15)

Xt = np.concatenate([X, X_unknown_add], axis = 0)
yt = np.concatenate([y, y_unknown_add], axis = 0)
    
X_train, X_test, y_train, y_test = train_test_split(Xt, yt, test_size=0.2, random_state=343)

print(X_train.shape, X_test.shape)